In [ ]:
%pip install ccxt
%pip install pandas

In [ ]:
import ccxt
import pandas as pd
import matplotlib.pyplot as plt
import time
from datetime import datetime, timedelta
import datetime

In [ ]:
def fetch_historical_data(exchange, symbol, timeframe, since):
    ohlcv = exchange.fetch_ohlcv(symbol, timeframe, since)
    df = pd.DataFrame(ohlcv, columns=['timestamp', 'open', 'high', 'low', 'close', 'volume'])
    df['timestamp'] = pd.to_datetime(df['timestamp'], unit='ms')
    df['date'] = df['timestamp'].dt.strftime('%m-%d-%y')
    df = df.tail(90) #used to select only the last 90 days of entries into the DF
    return df

In [ ]:
#   api key stuff
api_key = 'Z6bH9gXJ7UllNhBuj0'
api_secret = 'gtajv8bMci1T3BIEmTMzgS4PQU5IECuFWqKz'
exchange = ccxt.bybit({'apiKey': api_key, 'secret': api_secret})

#   asset and timeframe
symbol = 'BTCUSDT'
timeframe = '5m'

In [ ]:
# get today's date and subtract 90 days
day_90 = datetime.datetime.today() - datetime.timedelta(days=90)
since_timestamp = int(day_90.timestamp()) #convert 90 days to a timestamp 

#weekdays_data = pd.bdate_range(day_90, datetime.datetime.today())

# Fetch historical dataa
historical_data = exchange.fetch_ohlcv(symbol, timeframe, since_timestamp)

# Convert data to DataFrame
df = pd.DataFrame(historical_data, columns=['timestamp', 'open', 'high', 'low', 'close', 'volume'])
df['timestamp'] = pd.to_datetime(df['timestamp'], unit='ms')

# Filter data for weekdays and weekends
weekdays_data = df[df['timestamp'].dt.weekday < 5]  # Monday to Friday
weekends_data = df[df['timestamp'].dt.weekday >= 5]  # Saturday and Sunday

print(weekdays_data)

In [ ]:
# Plot for weekdays
plt.figure(figsize=(12, 6))
plt.plot(weekdays_data['timestamp'], weekdays_data['close'], label='BTC Price - Weekdays')
plt.title('BTC Price - Weekdays')
plt.xlabel('Timestamp')
plt.ylabel('Price (USD)')
plt.legend()
plt.grid(True)
plt.show()

# Plot for weekends
plt.figure(figsize=(12, 6))
plt.plot(weekends_data['timestamp'], weekends_data['close'], label='BTC Price - Weekends', color='orange')
plt.title('BTC Price - Weekends')
plt.xlabel('Timestamp')
plt.ylabel('Price (USD)')
plt.legend()
plt.grid(True)
plt.show()